# Search for Text in 10-K Footnotes with the Calcbench Text Search API and Save the Disclosures to Text Files
## Or, How to fill your hard-drive with disclosures that mention crypto-currency

Use the Calcbench text search API to search for terms in disclosures.  More about the API @ https://www.calcbench.com/home/api, get the Calcbench Python API client @ https://github.com/calcbench/python_api_client.

In [ ]:
%pip install calcbench-api-client[Backoff,BeautifulSoup] tqdm python-slugify[unidecode]

In [1]:
import os
import warnings
from pathlib import Path, PureWindowsPath
from slugify import slugify
import calcbench as cb
from tqdm.notebook import tqdm

cb.enable_backoff(giveup=lambda e: e.response.status_code == 404)

In [3]:
# Use Lucene query syntax, http://lucene.apache.org/core/2_9_4/queryparsersyntax.html
search_term = "crypto"

In [4]:
with tqdm() as progress_bar:
    search_results = list(
        cb.disclosure_search(
            full_text_search_term=search_term,
            entire_universe=True,
            all_history=True,
            progress_bar=progress_bar,
        )
    )

0it [00:00, ?it/s]

In [ ]:
for number, disclosure in tqdm(enumerate(search_results)):
    try:
        contents = disclosure.get_contents_text()
    except Exception as e:
        warnings.warn(f"error getting contents for {disclosure} {e}")
    else:
        file_name = f"{number}-{disclosure.ticker}-{disclosure.fiscal_year}-{disclosure.fiscal_period}-{disclosure.description}"
        file_name = (
            slugify(file_name[:250]) + ".txt"  # Max file length on Window is 260
        )
        file_path = os.path.join(
            Path.home(),
            "Downloads",
            "crypto_disclosures",
            file_name,
        )
        with open(file_path, "wb") as output:
            contents = contents.encode()
            output.write(contents)